# Simple call

In [23]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv(override=True)

messages = [{"role":"user", "content":"Give me 1 sentence differentiating between meiosis and mitosis."}]

# OpenAI
client = OpenAI()
response = client.chat.completions.create(
    messages=messages,
    model='gpt-4.1-nano'
)
result = (f"From gpt-4.1-nano: {response.choices[0].message.content}")
display(Markdown(result))

# Other
client = OpenAI(api_key=os.getenv('DEEPSEEK_API_KEY'), base_url="https://api.deepseek.com/v1")
response = client.chat.completions.create(
    messages=messages,
    model='deepseek-chat'
)
result = f"From deepseek-chat: {response.choices[0].message.content}"
display(Markdown(result))

From gpt-4.1-nano: Meiosis is a cell division process that reduces the chromosome number by half to produce gametes, while mitosis results in two genetically identical diploid daughter cells for growth and repair.

From deepseek-chat: Mitosis produces two genetically identical diploid cells for growth and repair, while meiosis creates four genetically unique haploid cells specifically for sexual reproduction.

# Agent

In [9]:
from agents import Agent, Runner
from agents import AsyncOpenAI, OpenAIResponsesModel, trace
import os

In [2]:
instruction = "You generate question about science that can be solved by college graduates."
agent = Agent(name="Science Question Agent", instructions=instruction, model='gpt-4.1-nano')
science_question_tool = agent.as_tool(tool_name="question_maker_science", tool_description="Ask a science question.")

In [3]:
instruction = "You generate question about art history that can be solved by college graduates."
agent = Agent(name='Art Question Agent', instructions=instruction, model='gpt-4.1-nano')
art_question_tool = agent.as_tool(tool_name='question_maker_art', tool_description="Ask an art question.")

In [10]:
instruction = "You provide answers to science and art history questions. Your knowledge is a college graduate level."
deepseek_client = AsyncOpenAI(api_key=os.getenv('DEEPSEEK_API_CALL'), base_url='https://api.deepseek.com/v1')
deepseek_model = OpenAIResponsesModel(model='deepseek-chat', openai_client=deepseek_client)
agent = Agent(name='Answer Provider Agent', instructions=instruction, model=deepseek_model)
answer_tool = agent.as_tool(tool_name='solution_provider', tool_description="Give an answer.")

In [11]:
lecturer_instructions = """
You give a question and answer. Your goal is to select

Follow these steps carefully:
1. Generate questions: Use question_maker_art and qeustion_maker_science tools to generate 2 different questions.
2. Evaluate: Provide answers to the 2 questions by using solution_provider tool.
3. Report: Publish the questions and answers.

"""
lecturer = Agent(
    name="Lecturer",
    instructions=lecturer_instructions,
    tools = [science_question_tool, art_question_tool, answer_tool],
    model='gpt-4.1-nano'
)

message = "Give me a question and answer so that I can learn something new today."

with trace('QandA2'):
    result = await Runner.run(lecturer, message)

    print(result.final_output)

Error getting response: Error code: 404 - {'error_msg': 'Not Found. Please check the configuration.'}. (request_id: None)
Error getting response: Error code: 404 - {'error_msg': 'Not Found. Please check the configuration.'}. (request_id: None)


I encountered an issue while trying to retrieve the answers. However, I can provide you with the answers:

**Art Question:** The development of linear perspective during the Renaissance allowed artists to create the illusion of depth on a flat surface, making their paintings more realistic. It introduced a systematic way to depict spatial relationships and scale, revolutionizing visual representation and influencing countless artists and artworks.

**Science Question:** Photosynthesis converts light energy into chemical energy through a process that occurs in two main stages: the light-dependent reactions and the Calvin cycle. In the first stage, light energy is absorbed by chlorophyll and used to split water molecules, releasing oxygen and producing ATP and NADPH. In the second stage, these energy carriers help convert carbon dioxide into glucose, storing chemical energy for the plant.


# Agent (Async)